In [1]:
import pandas as pd


# Chargement des fichiers CSV
benin_data = pd.read_csv("../data/dataBell_DNS Dataset/features_domain_benign_csv.csv")
malware_data = pd.read_csv("../data/Bell_DNS Dataset/features-domain_Malware.csv")
phishing_data = pd.read_csv("../data/Bell_DNS Dataset/features-domain_phishing.csv")
spam_data = pd.read_csv("../data/dataBell_DNS Dataset/features-domain_Spam.csv")

# Ajout de la colonne 'Class' 
benin_data['Class'] = 'Benin'
malware_data['Class'] = 'Malware'
phishing_data['Class'] = 'Phishing'
spam_data['Class'] = 'Spam'


# Concaténation de tous les ensembles de données
all_data = pd.concat([benin_data, malware_data, phishing_data, spam_data], ignore_index=True)


In [2]:
# Spécifier la colonne cible je doit le faire avant l'encodage
y = all_data['Class']

In [3]:
print(all_data.dtypes)


subdomain                  float64
tld                         object
sld                         object
len                          int64
numeric_percentage         float64
char_distribution           object
entropy                    float64
1gram                       object
2gram                       object
3gram                       object
longest_word                object
distance_from_bad_words     object
typos                       object
obfuscate_at_sign           object
dec_8                        int64
dec_32                       int64
oc_8                         int64
oc_32                        int64
hex_8                        int64
hex_32                       int64
puny_coded                   int64
shortened                   object
Domain_Name                 object
Registrar                   object
Registrant_Name             object
Creation_Date_Time          object
Emails                      object
Domain_Age                  object
Organization        

In [ ]:
print(all_data['Class'].unique())

print(all_data.info())

all_data.isnull().sum().sort_values(ascending=False)

print(all_data.shape)




In [5]:
from sklearn.impute import SimpleImputer

# Concaténation de tous les ensembles de données
all_data = pd.concat([benin_data, malware_data, phishing_data, spam_data], ignore_index=True)

# Convertir les variables catégorielles en variables numériques
all_data_encoded = pd.get_dummies(all_data, columns=['Class'], drop_first=True)

# Exclure les colonnes non numériques


#X = all_data_encoded.select_dtypes(include='number') ##### OK mais je ne veux pas faire ainsi 

X_numerical = all_data.select_dtypes(include=['int64', 'float64'])


# Vérifier les colonnes avec des valeurs manquantes
missing_columns = X_numerical.columns[X_numerical.isnull().any()]

print("Missing value : {}", X_numerical[missing_columns])

# Imputation des valeurs manquantes
if not missing_columns.empty:
    imputer = SimpleImputer(strategy='mean')  # Vous pouvez également utiliser 'median' ou 'most_frequent'
    X_numerical[missing_columns] = imputer.fit_transform(X_numerical[missing_columns])



#apres remplissage avec la valuer moyenne    
print("Missing value : {}", all_data[missing_columns])


Missing value : {}        subdomain  Alexa_Rank  Page_Rank
0            0.0         1.0        NaN
1            0.0         1.0        NaN
2            1.0         1.0        NaN
3            1.0         1.0        NaN
4            0.0         7.0        NaN
...          ...         ...        ...
50154        0.0        -1.0        NaN
50155        0.0        -1.0        NaN
50156        0.0        -1.0        NaN
50157        0.0        -1.0        NaN
50158        0.0        -1.0        NaN

[50159 rows x 3 columns]
Missing value : {}        subdomain  Alexa_Rank  Page_Rank
0            0.0         1.0        NaN
1            0.0         1.0        NaN
2            1.0         1.0        NaN
3            1.0         1.0        NaN
4            0.0         7.0        NaN
...          ...         ...        ...
50154        0.0        -1.0        NaN
50155        0.0        -1.0        NaN
50156        0.0        -1.0        NaN
50157        0.0        -1.0        NaN
50158        0.0

In [ ]:
###################### Amelioration du code plus haut ######################

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# 2. Vérification des colonnes vides
if X_numerical.isnull().any().any():
    # Imputer les valeurs manquantes pour les caractéristiques numériques
    numerical_imputer = SimpleImputer(strategy='mean')
    X_numerical_imputed = pd.DataFrame(numerical_imputer.fit_transform(X_numerical), columns=X_numerical.columns)

    #prepocessing des features numeriques soit  avec le  LabelEncoder soit le MinMaxScaler()

    # Création d'un scaler
    scaler = MinMaxScaler()

    # Ajustement du scaler aux données
    scaler.fit(X_numerical_imputed)

    # Transformation des fonctionnalités numériques
    scaled_numeric_features = scaler.transform(X_numerical_imputed)


    print(scaled_numeric_features)

    # Apres transformation Création d' un DataFrame à partir des valeurs transformées

    scaled_df = pd.DataFrame(scaled_numeric_features, columns=X_numerical_imputed.columns)

    # Afficher le DataFrame avec les valeurs transformées
    print("Après transformation")
    print(scaled_df)

    total_size = scaled_df.shape
    print("Taille totale des caractéristiques numériques après transformations :", total_size)
else:
    print("Pas de valeurs manquantes dans les caractéristiques numériques. Aucune imputation nécessaire.")


In [6]:
print(X_numerical.shape)

(50159, 13)


In [8]:
num_features = X_numerical.shape[1]
print("Nombre total de caractéristiques numeriques (car les autres ont été ignorer):", num_features)


#donc dans notre fonction il faut passer un nombre de caracteristiques inferieur ou egale a 13

Nombre total de caractéristiques numeriques (car les autres ont été ignorer): 13


In [9]:
#### j'affiche voir la variance 


X_numerical.var()

subdomain             2.468767e-01
len                   5.041374e+01
numeric_percentage    4.878897e+01
entropy               3.723222e-01
dec_8                 0.000000e+00
dec_32                6.559620e-03
oc_8                  0.000000e+00
oc_32                 7.974164e-05
hex_8                 0.000000e+00
hex_32                0.000000e+00
puny_coded            4.682994e-03
Alexa_Rank            3.907441e+12
Page_Rank             0.000000e+00
dtype: float64

In [10]:
import pandas as pd
import numpy as np

# ... (votre code pour charger et prétraiter les données)

# Calcul de l'entropie
def calculate_entropy(y):
    unique_classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

# Spécifier la colonne cible
y = all_data['Class']

# Calcul de l'entropie pour la colonne cible
entropy_y = calculate_entropy(y)
print("Entropie de la colonne cible:", entropy_y)


Entropie de la colonne cible: 1.7561017914049486


In [12]:
# Calcul de l'entropie de la caractéristique X
def calculate_feature_entropy(X_feature, y):
    unique_values, counts = np.unique(X_feature, return_counts=True)
    entropy_feature = 0

    for value, count in zip(unique_values, counts):
        indices = X_feature == value
        entropy_feature += (count / len(y)) * calculate_entropy(y[indices])

    return entropy_feature

# Calcul de l'information intrinsèque de la caractéristique X
def calculate_intrinsic_information(X_feature):
    unique_values, counts = np.unique(X_feature, return_counts=True)
    intrinsic_info = 0

    for count in counts:
        intrinsic_info += (count / len(X_feature)) * np.log2(count / len(X_feature))

    return -intrinsic_info

# Calcul du Gain Ratio
def calculate_gain_ratio(X_feature, y):
    entropy_X = calculate_feature_entropy(X_feature, y)
    intrinsic_info_X = calculate_intrinsic_information(X_feature)

    gain = entropy_y - entropy_X
    gain_ratio = gain / intrinsic_info_X

    return gain_ratio

# Exemple d'utilisation
#X_feature_example = X['nom_de_la_caracteristique']  # Remplacez 'nom_de_la_caracteristique' par le nom de votre caractéristique

X_feature_example = X_numerical['Alexa_Rank']

gain_ratio_example = calculate_gain_ratio(X_feature_example, y)
print("Gain Ratio de la caractéristique:", gain_ratio_example)


Gain Ratio de la caractéristique: 0.11313698610951292
